In [1]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from text_cleaner import TextCleaner # Đảm bảo file text_cleaner.py ở cùng thư mục hoặc trong PYTHONPATH
import os

In [3]:
%pip install py_vncorenlp 


  Using cached py_vncorenlp-0.1.4.tar.gz (3.9 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4392 sha256=70b02fb984584ad296f560afe4cb4bdecf3530d8c5f1d76f70868218c8e41ba4
  Stored in directory: c:\users\mai xuan truong\appdata\local\pip\cache\wheels\d5\d9\bf\62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from text_cleaner import TextCleaner # Đảm bảo file text_cleaner.py ở cùng thư mục hoặc trong PYTHONPATH
import os

In [5]:
# --- 0. Configuration ---
# THAY ĐỔI ĐƯỜNG DẪN NÀY CHO PHÙ HỢP VỚI THƯ MỤC BẠN ĐÃ LƯU MÔ HÌNH
# Ví dụ:
# MODEL_SAVE_DIR = r'E:\LEARN_5\XuLyNgonNguTuNhien\project\models_spacy'
# HOẶC
MODEL_SAVE_DIR = r'E:\LEARN_5\XuLyNgonNguTuNhien\project\models' # Ví dụ nếu bạn dùng model với Word2Vec
# HOẶC
# MODEL_SAVE_DIR = r'E:\LEARN_5\XuLyNgonNguTuNhien\project\models' # Thư mục gốc ban đầu

print(f"Sử dụng MODEL_SAVE_DIR: {MODEL_SAVE_DIR}")
if not os.path.exists(MODEL_SAVE_DIR):
    print(f"LỖI: Thư mục MODEL_SAVE_DIR '{MODEL_SAVE_DIR}' không tồn tại. Vui lòng kiểm tra lại đường dẫn.")
    exit()

TOKENIZER_PATH = os.path.join(MODEL_SAVE_DIR, 'tokenizer.pkl')
LABEL_ENCODER_PATH = os.path.join(MODEL_SAVE_DIR, 'label_encoder.pkl')
MAX_LEN_PATH = os.path.join(MODEL_SAVE_DIR, 'max_len.pkl')
BIGRU_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, 'bigru_model.keras')
BILSTM_MODEL_PATH = os.path.join(MODEL_SAVE_DIR, 'bilstm_model.keras')

# Paths for TextCleaner
STOPWORDS_FILE = 'E:\\LEARN_5\\XuLyNgonNguTuNhien\\project\\crawl\\stopwords.txt'
ABBREVIATIONS_FILE = 'E:\\LEARN_5\\XuLyNgonNguTuNhien\\project\\crawl\\acronym.txt'
VNCORENLP_MODEL_DIR = 'E:/LEARN_5/XuLyNgonNguTuNhien/vncorenlp'







Sử dụng MODEL_SAVE_DIR: E:\LEARN_5\XuLyNgonNguTuNhien\project\models


In [6]:
# --- 1. Khởi tạo TextCleaner (Tương tự ví dụ của bạn) ---
print("--- Khởi tạo TextCleaner ---")
cleaner_instance = None
try:
    if not os.path.exists(STOPWORDS_FILE): print(f"CẢNH BÁO: File stopwords '{STOPWORDS_FILE}' không tồn tại.")
    if not os.path.exists(ABBREVIATIONS_FILE): print(f"CẢNH BÁO: File viết tắt '{ABBREVIATIONS_FILE}' không tồn tại.")
    if not os.path.exists(VNCORENLP_MODEL_DIR): print(f"CẢNH BÁO: Thư mục VnCoreNLP '{VNCORENLP_MODEL_DIR}' không tồn tại.")

    # Kiểm tra xem instance đã tồn tại chưa (mặc dù trong script độc lập này thì không cần thiết lắm)
    if 'cleaner_instance' not in globals() or cleaner_instance is None:
        cleaner_instance = TextCleaner(STOPWORDS_FILE, ABBREVIATIONS_FILE, VNCORENLP_MODEL_DIR)
        print("Đã tạo instance mới của TextCleaner.")
    else:
        print("Instance của TextCleaner đã tồn tại. Bỏ qua khởi tạo.")
except Exception as e:
    print(f"Lỗi khi khởi tạo TextCleaner: {e}. Không thể làm sạch văn bản.")
    # Để script có thể tiếp tục (nếu muốn test phần tải model), không exit() ở đây
    # nhưng hàm dự đoán sẽ không hoạt động nếu cleaner_instance là None.


--- Khởi tạo TextCleaner ---
Đã tạo instance mới của TextCleaner.


In [7]:
# --- 2. Tải các thành phần đã lưu ---
print("\n--- Tải các thành phần đã lưu ---")
tokenizer = None
label_encoder = None
max_len = None
bigru_model = None
bilstm_model = None

try:
    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer = pickle.load(handle)
    print(f"Đã tải tokenizer từ {TOKENIZER_PATH}.")
except Exception as e:
    print(f"Lỗi khi tải tokenizer: {e}.")

try:
    with open(LABEL_ENCODER_PATH, 'rb') as handle:
        label_encoder = pickle.load(handle)
    print(f"Đã tải label encoder từ {LABEL_ENCODER_PATH}.")
except Exception as e:
    print(f"Lỗi khi tải label encoder: {e}.")

try:
    with open(MAX_LEN_PATH, 'rb') as handle:
        max_len = pickle.load(handle)
    print(f"Đã tải max_len: {max_len} từ {MAX_LEN_PATH}")
except Exception as e:
    print(f"Lỗi khi tải max_len: {e}.")

try:
    bigru_model = load_model(BIGRU_MODEL_PATH)
    print(f"Đã tải mô hình BiGRU từ {BIGRU_MODEL_PATH}.")
except Exception as e:
    print(f"Lỗi khi tải mô hình BiGRU: {e}.")

try:
    bilstm_model = load_model(BILSTM_MODEL_PATH)
    print(f"Đã tải mô hình BiLSTM từ {BILSTM_MODEL_PATH}.")
except Exception as e:
    print(f"Lỗi khi tải mô hình BiLSTM: {e}.")


--- Tải các thành phần đã lưu ---
Đã tải tokenizer từ E:\LEARN_5\XuLyNgonNguTuNhien\project\models\tokenizer.pkl.
Đã tải label encoder từ E:\LEARN_5\XuLyNgonNguTuNhien\project\models\label_encoder.pkl.
Đã tải max_len: 200 từ E:\LEARN_5\XuLyNgonNguTuNhien\project\models\max_len.pkl
Đã tải mô hình BiGRU từ E:\LEARN_5\XuLyNgonNguTuNhien\project\models\bigru_model.keras.
Đã tải mô hình BiLSTM từ E:\LEARN_5\XuLyNgonNguTuNhien\project\models\bilstm_model.keras.


In [22]:
# --- 3. Dữ liệu mẫu và làm sạch (Tương tự ví dụ của bạn) ---
print("\n--- Chuẩn bị dữ liệu mẫu ---")
sample_title = "Các ông lớn công nghệ giữa ngã ba đường vì AI"
sample_description = "Các hãng công nghệ hàng đầu như Apple, Facebook, Google được đánh giá bắt đầu loay hoay tìm cách thay đổi trong bối cảnh AI phát triển thần tốc."
sample_content = "AI đang được coi là \"kẻ quấy phá\" ở Thung lũng Silicon, đặc biệt với các tập đoàn lâu đời như Apple (49 năm), Google (26 năm), Meta (21 năm) hay Amazon (31 năm) \"Công bằng mà nói, chưa Big Tech nào gặp khó khăn. Họ vẫn là trụ cột của nước Mỹ với tổng vốn hóa thị trường trên 7.000 tỷ USD\", WSJ bình luận. \"Tuy nhiên, họ đều đang cùng nhau đứng ở ngã ba đường và tiến thoái lưỡng nan, điều cuốn sách kinh doanh kinh điển The Innovator's Dilemma từng đề cập\" \"The Innovator's Dilemma (Thách thức sáng tạo) của tác giả Clayton Christensen giải thích cách các sản phẩm, dịch vụ mới \"lật đổ\" cái cũ. Cuốn sách nhắc đến thuật ngữ disruption, ý nói các công ty lớn và thành công có thể thất bại trước sự xuất hiện công ty nhỏ sở hữu công nghệ, quy trình đột phá. Giống như Internet từng \"thanh lọc\" nhiều doanh nghiệp truyền thống, AI cũng có thể khiến các ông lớn công nghệ hiện nay phải thay đổi hoặc chung số phận\" \"Theo Bloomberg, tại phiên tòa chống độc quyền của Bộ Tư pháp đối với Google tuần trước, Eddy Cue, Phó chủ tịch mảng dịch vụ của Apple, cho biết lượng truy vấn tìm kiếm của Google trên thiết bị công ty lần đầu tiên giảm sau 20 năm vào tháng trước. Dù Google phủ nhận, CNN nhận định tiết lộ của Eddy Cue cho thấy người dùng đang dần chuyển sang hỏi đáp thông tin bằng chatbot AI. Năm ngoái, công ty nghiên cứu thị trường Gartner ước tính tìm kiếm trực tuyến kiểu truyền thông sẽ giảm 25% vào năm 2026 do AI trở nên thịnh hành\" \"CEO Apple Tim Cook cũng đang phải trấn an nhà đầu tư trước sự chậm trễ triển khai AI trên iPhone. \"Chúng tôi cần thêm thời gian để hoàn thành đúng theo tiêu chuẩn chất lượng cao nhất\", ông nói trong cuộc họp trước giới đầu tư tuần trước về Apple Intelligence\" \"Trong khi đó, Mark Zuckerberg, CEO Meta, cho biết người dùng đang giảm việc chia sẻ với bạn bè trên Facebook. Theo giới chuyên gia, điều này một phần do mạng xã hội hơn 20 năm tuổi trở nên \"già cỗi\" trước sự mới mẻ của TikTok với thuật toán AI đề xuất nội dung\" \"Một số đã triển khai giải pháp riêng, như Meta tung ra chatbot Meta AI, Google có mô hình Gemini còn Microsoft phát triển Copilot. Tuy nhiên, chưa ai tìm ra \"công thức chiến thắng\". Họ đang ở ngã ba đường: có quy mô quá lớn, không thể xóa bỏ hoàn toàn mô hình cũ để mạnh dạn thử nghiệm những thứ mới mẻ như các startup nhỏ\" \"Bạn không thể gắn bó với một khuôn khổ hoạt động hoặc một cách làm cụ thể nào đó trong thời AI\", Karl Mozurkewich, kiến trúc sư trưởng tại công ty điện toán đám mây Valdi, viết trên blog\" \"Điều này mang lại hy vọng cho những người như Sarah Guo, nhà đầu tư mạo hiểm trẻ ở Thung lũng Silicon. Cô đang tìm kiếm startup AI hấp dẫn tiếp theo - những công ty có thể hạ bệ ông lớn. \"Với công ty đã thành danh, việc sáng tạo sản phẩm mới sẽ đầy rủi ro\", Guo nói trong podcast Bold Names phát sóng tuần trước\" \"Đây là điều tôi thích nhất về AI\", Theo Brown, nhà phát triển Ping, nói. \"Nó biến những thứ trước đây ngành công nghệ coi là siêu giá trị và thiêng liêng thành thứ cực kỳ rẻ và dễ vứt bỏ\" \"Không phải hãng công nghệ lớn nào cũng bị tác động theo hướng tiêu cực. Nvidia, ra đời cách đây 32 năm, phát triển bùng nổ nhờ cơn khát GPU dùng để huấn luyện AI. Tuy nhiên, sự xuất hiện của các mô hình giá rẻ như DeepSeek cũng đặt ra câu hỏi liệu có cần đầu tư quá nhiều tiền vào chip AI như hiện nay\" \"Tất cả đang trong tình huống tiến thoái lưỡng nan, dù chưa có ai thực sự gặp khủng hoảng\", WSJ bình luận."
cleaned_sample_text = ""
if cleaner_instance:
    try:
        title_c, desc_c, content_c = cleaner_instance.process_text(sample_title, sample_description, sample_content)
        cleaned_sample_text = title_c + " " + desc_c + " " + content_c
        print("Dữ liệu mẫu đã được làm sạch.")
        print(f"Văn bản đã làm sạch (200 ký tự đầu): \"{cleaned_sample_text[:200]}...\"")
    except Exception as e:
        print(f"Lỗi khi làm sạch dữ liệu mẫu: {e}")
else:
    print("TextCleaner không khả dụng, không thể làm sạch dữ liệu mẫu.")




--- Chuẩn bị dữ liệu mẫu ---
Dữ liệu mẫu đã được làm sạch.
Văn bản đã làm sạch (200 ký tự đầu): "công_nghệ ngã đường trí_tuệ_nhân_tạo hãng công_nghệ hàng_đầu apple facebook google loay_hoay bối_cảnh trí_tuệ_nhân_tạo phát_triển thần_tốc trí_tuệ_nhân_tạo coi kẻ quấy_phá thung_lũng silicon tập_đoàn ..."


In [23]:
# --- 4. Hàm dự đoán (Tương tự ví dụ của bạn, nhưng cho RNN) ---
def predict_new_text_rnn(cleaned_text_input, model_to_use, model_name_str,
                         tokenizer_loaded, label_encoder_loaded, max_len_loaded):
    # Kiểm tra các thành phần cần thiết
    if model_to_use is None:
        print(f"Mô hình {model_name_str} chưa được tải hoặc có lỗi. Bỏ qua dự đoán.")
        return
    if tokenizer_loaded is None:
        print("Tokenizer chưa được tải. Bỏ qua dự đoán.")
        return
    if label_encoder_loaded is None:
        print("Label encoder chưa được tải. Bỏ qua dự đoán.")
        return
    if max_len_loaded is None:
        print("Max_len chưa được tải. Bỏ qua dự đoán.")
        return
    if not cleaned_text_input or not cleaned_text_input.strip():
        print("Văn bản đầu vào (đã làm sạch) rỗng. Không thể dự đoán.")
        return

    print(f"\n--- Dự đoán với {model_name_str} cho văn bản mới ---")
    print(f"Văn bản đầu vào (200 ký tự đầu): \"{cleaned_text_input[:200]}...\"")

    # Tokenize and Pad
    try:
        sequence = tokenizer_loaded.texts_to_sequences([cleaned_text_input])
        padded_sequence = pad_sequences(sequence, maxlen=max_len_loaded, padding='post', truncating='post')
    except Exception as e:
        print(f"Lỗi trong quá trình tokenize hoặc padding: {e}")
        return

    # Predict
    try:
        prediction_proba = model_to_use.predict(padded_sequence)
        predicted_class_index = np.argmax(prediction_proba, axis=1)[0]
        predicted_class_label = label_encoder_loaded.inverse_transform([predicted_class_index])[0]
    except Exception as e:
        print(f"Lỗi trong quá trình dự đoán của mô hình: {e}")
        return

    print(f"Chủ đề dự đoán: {predicted_class_label}")
    print("Xác suất dự đoán cho từng chủ đề:")
    if hasattr(label_encoder_loaded, 'classes_'):
        for i, topic_class_name in enumerate(label_encoder_loaded.classes_):
            print(f"  - {topic_class_name}: {prediction_proba[0][i]:.4f}")
    else:
        print("Không thể lấy tên các lớp từ label_encoder.")



In [24]:
# --- 5. Thực hiện dự đoán (Tương tự ví dụ của bạn) ---
print("\n--- Thực hiện dự đoán cho dữ liệu mẫu ---")
if cleaned_sample_text: # Chỉ dự đoán nếu văn bản đã được làm sạch
    if bigru_model:
        predict_new_text_rnn(cleaned_sample_text, bigru_model, "BiGRU",
                             tokenizer, label_encoder, max_len)
    else:
        print("Mô hình BiGRU không khả dụng để dự đoán.")

    if bilstm_model:
        predict_new_text_rnn(cleaned_sample_text, bilstm_model, "BiLSTM",
                             tokenizer, label_encoder, max_len)
    else:
        print("Mô hình BiLSTM không khả dụng để dự đoán.")

    if not bigru_model and not bilstm_model:
        print("Không có mô hình RNN nào được tải thành công để thực hiện dự đoán.")
elif cleaner_instance is None:
    print("Không thể dự đoán vì TextCleaner không được khởi tạo.")
else:
    print("Không thể dự đoán vì dữ liệu mẫu chưa được làm sạch (có thể do lỗi ở TextCleaner).")





--- Thực hiện dự đoán cho dữ liệu mẫu ---

--- Dự đoán với BiGRU cho văn bản mới ---
Văn bản đầu vào (200 ký tự đầu): "công_nghệ ngã đường trí_tuệ_nhân_tạo hãng công_nghệ hàng_đầu apple facebook google loay_hoay bối_cảnh trí_tuệ_nhân_tạo phát_triển thần_tốc trí_tuệ_nhân_tạo coi kẻ quấy_phá thung_lũng silicon tập_đoàn ..."
1/1 [==============================] - 0s 99ms/step
Chủ đề dự đoán: Chuyển đổi số
Xác suất dự đoán cho từng chủ đề:
  - AI: 0.2140
  - Bóng đá: 0.0000
  - Chuyển đổi số: 0.6300
  - Chân dung: 0.0000
  - Chính trị: 0.0000
  - Chứng khoán: 0.0003
  - Các môn thể thao khác: 0.0001
  - Du học: 0.0000
  - Dân sinh: 0.0000
  - Ebank: 0.0001
  - Giao thông: 0.0000
  - Giáo dục 4.0: 0.0002
  - Giải trí giới sao: 0.0002
  - Giải trí làm đẹp: 0.0000
  - Giải trí nhạc: 0.0000
  - Giải trí phim: 0.0091
  - Giải trí thời trang: 0.0020
  - Hàng hóa: 0.0002
  - Hậu trường thể thao: 0.0001
  - Học tiếng anh: 0.0000
  - Hồ sơ phá án: 0.0048
  - Kinh doanh doanh nghiệp: 0.0017
  - Kin

In [11]:
# Ví dụ với dữ liệu thứ hai (bạn có thể thêm)
print("\n--- Dự đoán cho dữ liệu mẫu thứ hai (tùy chọn) ---")
sample_title_2 = "Giá vàng trong nước bất ngờ tăng mạnh"
sample_description_2 = "Giá vàng SJC và các loại vàng khác đều điều chỉnh tăng sau nhiều phiên đi ngang."
sample_content_2 = "Sáng nay, các công ty kinh doanh vàng bạc đá quý lớn đồng loạt niêm yết giá vàng SJC tăng từ 200.000 đến 300.000 đồng mỗi lượng. Giá vàng nhẫn cũng ghi nhận mức tăng tương tự. Sự biến động này được cho là do ảnh hưởng từ thị trường vàng thế giới và tỷ giá USD/VND. Nhiều nhà đầu tư tỏ ra quan tâm và tìm kiếm cơ hội giao dịch."

cleaned_sample_text_2 = ""
if cleaner_instance:
    try:
        title_c2, desc_c2, content_c2 = cleaner_instance.process_text(sample_title_2, sample_description_2, sample_content_2)
        cleaned_sample_text_2 = title_c2 + " " + desc_c2 + " " + content_c2
        print("Dữ liệu mẫu 2 đã được làm sạch.")
    except Exception as e:
        print(f"Lỗi khi làm sạch dữ liệu mẫu 2: {e}")

if cleaned_sample_text_2:
    if bigru_model:
        predict_new_text_rnn(cleaned_sample_text_2, bigru_model, "BiGRU",
                             tokenizer, label_encoder, max_len)
    if bilstm_model:
        predict_new_text_rnn(cleaned_sample_text_2, bilstm_model, "BiLSTM",
                             tokenizer, label_encoder, max_len)

print("\n--- Kết thúc chương trình dự đoán ---")


--- Dự đoán cho dữ liệu mẫu thứ hai (tùy chọn) ---
Dữ liệu mẫu 2 đã được làm sạch.

--- Dự đoán với BiGRU cho văn bản mới ---
Văn bản đầu vào (200 ký tự đầu): "giá vàng giá vàng sjc vàng điều_chỉnh phiên đi ngang công_ty kinh_doanh vàng_bạc đá_quý đồng_loạt niêm_yết giá vàng sjc đồng giá vàng nhẫn ghi_nhận tương_tự biến_động ảnh_hưởng thị_trường vàng thế_giớ..."
1/1 [==============================] - 0s 88ms/step
Chủ đề dự đoán: Hàng hóa
Xác suất dự đoán cho từng chủ đề:
  - AI: 0.0000
  - Bóng đá: 0.0000
  - Chuyển đổi số: 0.0001
  - Chân dung: 0.0000
  - Chính trị: 0.0000
  - Chứng khoán: 0.0000
  - Các môn thể thao khác: 0.0000
  - Du học: 0.0000
  - Dân sinh: 0.0000
  - Ebank: 0.0002
  - Giao thông: 0.0001
  - Giáo dục 4.0: 0.0000
  - Giải trí giới sao: 0.0000
  - Giải trí làm đẹp: 0.0000
  - Giải trí nhạc: 0.0000
  - Giải trí phim: 0.0000
  - Giải trí thời trang: 0.0000
  - Hàng hóa: 0.9973
  - Hậu trường thể thao: 0.0000
  - Học tiếng anh: 0.0000
  - Hồ sơ phá án: 0.0000
  - Kin